PAC. Pràctica 2. Tipologia i cicle de vida de les dades. Universitat Oberta de Catalunya.

J. de Curtò i DíAz & I. de Zarzà i Cubero. decurto@uoc.edu dezarza@uoc.edu

# StyleGan2-ADA trained on data from NASA Perseverance

<p align="center">
    <a
    href="https://github.com/decurtoidiaz/drcyz"
    target="_blank"
    rel="noopener noreferrer">
        <img
        alt="Neural Mars: Dr CyZ"
        width="450" height="450"
        src="https://github.com/decurtoidiaz/drcyz/blob/main/dr_cyz+.png?raw=true">
    </a>
</p>





StyleGan2-ADA trained on a subset of images from NASA Perseverance (DrCyZ).

Adapt folder paths accordingly. Data is available at:

CyZ: MARS Space Exploration Dataset.

https://github.com/decurtoidiaz/cyz

DrCyZ: Techniques for analyzing and extracting useful information from CyZ.

https://github.com/decurtoidiaz/drcyz

c@decurto.be z@dezarza.be

Original source code from:

https://colab.research.google.com/github/dvschultz/ml-art-colabs/blob/master/Stylegan2_ada_Custom_Training.ipynb

adapted and modified by De Curtò i DíAz & De Zarzà i Cubero.

1.   [Install StyleGAN2-ADA on your Google Drive](#scrollTo=5YcUMPQp6ipP)
2.   [Train a custom model](#scrollTo=Ti11YiPAiQpb)
3.   [Generate images from pre-trained model](#scrollTo=f0A9ZNtferpk)
4.   [Latent space exploration](#scrollTo=5yG1UyHXXqsO)


## Install StyleGAN2-ADA on your Google Drive

StyleGAN2-ADA only works with Tensorflow 1. Run the next cell before anything else to make sure we’re using TF1 and not TF2.


In [ ]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Fri Dec 31 14:27:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

Then, mount your Drive to the Colab notebook: 

In [ ]:
from google.colab import drive
from pathlib import Path

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.mount(str(drive_path))

Mounted at /content/drive


Finally, run this cell to install StyleGAN2-ADA on your Drive. If you’ve already installed the repository, it will skip the installation process and only check for updates. If you haven’t installed it, it will install all the necessary files. Beside, **in**, **out**, **datasets** and **training** folders are generated for data storage. Everything will be available on your Google Drive in the folder **StyleGAN2-ADA** even after closing this Notebook.

In [ ]:
stylegan2_repo_url  = 'https://github.com/dvschultz/stylegan2-ada' # or https://github.com/NVlabs/stylegan2-ada
project_path        = drive_path / 'MyDrive' / 'StyleGAN2-ADA'
stylegan2_repo_path = project_path / 'stylegan2-ada'

# Create project folder if inexistant
if not project_path.is_dir():
    %mkdir "{project_path}"
%cd "{project_path}"

for dir in ['in', 'out', 'datasets', 'training']:
    if not (project_path / dir).is_dir():
        %mkdir {dir}
if not (project_path / 'datasets' / 'source').is_dir():
    %mkdir "{project_path / 'datasets' / 'source'}"

 Download StyleGAN2-ada
!git config --global user.name "ArthurFDLR"
!git config --global user.email "arthfind@gmail.com"
if stylegan2_repo_path.is_dir():
    !git -C "{stylegan2_repo_path}" fetch origin
    !git -C "{stylegan2_repo_path}" checkout origin/main -- *.py
else:
    print("Install StyleGAN2-ADA")
    !git clone {stylegan2_repo_url}

/content/drive/MyDrive/StyleGAN2-ADA


## Train a custom model

Once you have installed StyleGAN2-ADA on your Google Drive and set up the working directory, you can upload your training dataset images in the associated folder.

In [ ]:
dataset_name = 'dr_cyz'
datasets_source_path = project_path / 'datasets' / 'source' / (dataset_name + '.zip')
if datasets_source_path.is_dir():
    print("Dataset ready for import.")
else:
    print('Upload your images dataset as {}'.format(datasets_source_path))

Upload your images dataset as /content/drive/MyDrive/StyleGAN2-ADA/datasets/source/dr_cyz.zip


Unfortunately, large datasets might exceed the Google Drive quota after a few training batches. Indeed, StyleGAN2 download datasets multiple times during training. You might have to import your dataset in the local storage session. However, large files cannot be copy/paste from Drive *(Input/Output error)*. 

Run this cell to download your zipped dataset from your Drive and unzip it in the local session.

In [ ]:
local_dataset_path = content_path / 'dataset'
if not local_dataset_path.is_dir():
    print("Importing dataset...")
    %mkdir "{local_dataset_path}"
    %cp -a "{project_path / 'datasets' / 'source' / (dataset_name + '.zip')}" "{local_dataset_path}"
    print("Zip file succesfuly imported")
else:
    print('Zip file allready imported')

import zipfile
with zipfile.ZipFile(str(local_dataset_path / (dataset_name + '.zip')), 'r') as zip_ref:
    zip_ref.extractall(str(local_dataset_path))
print('Extraction completed')

Importing dataset...
Zip file succesfuly imported
Extraction completed


### Convert dataset to .tfrecords

Next, we need to convert our image dataset to a format that StyleGAN2-ADA can read:`.tfrecords`.

This can take a while.

In [ ]:
local_original_images_path = local_dataset_path / 'c' 
local_images_path = local_dataset_path / 'dr_c_y_z_256'
local_dataset_path /= 'tfr_256'
!apt install imagemagick

if (local_dataset_path).is_dir():
    print('\N{Heavy Exclamation Mark Symbol} Dataset already created \N{Heavy Exclamation Mark Symbol}')
    print('Delete current dataset folder ({}) to regenerate tfrecords.'.format(local_dataset_path))
else:
    %mkdir "{local_images_path}"
    !mogrify -path "{local_images_path}" -resize 256x256! "{local_original_images_path / '*.png'}"
    %mkdir "{local_dataset_path}"
    !python "{stylegan2_repo_path / 'dataset_tool.py'}" create_from_images \
        "{local_dataset_path}" "{local_images_path}"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono g

There are numerous arguments to tune the training of your model. To obtain nice results, you will certainly have to experiment. Here are the most popular parameters:


*   *mirror:* Should the images be mirrored vertically?
*   *mirrory:* Should the images be mirrored horizontally?
*   *snap:* How often should the model generate image samples and a network pickle (.pkl file)?
*   *resume:* Network pickle to resume training from?

To see all the options, run the following ```help``` cell.

Please note that Google Colab Pro gives access to V100 GPUs, which drastically decreases (~3x) processing time over P100 GPUs.

In [ ]:
!python "{stylegan2_repo_path / 'train.py'}" --help

usage: train.py [-h] --outdir DIR [--gpus INT] [--snap INT] [--seed INT] [-n]
                --data PATH [--res INT] [--mirror BOOL] [--mirrory BOOL]
                [--use-raw BOOL] [--metrics LIST] [--metricdata PATH]
                [--cfg {auto,11gb-gpu,11gb-gpu-complex,24gb-gpu,24gb-gpu-complex,48gb-gpu,48gb-2gpu,stylegan2,paper256,paper512,paper1024,cifar,cifarbaseline,aydao}]
                [--lrate FLOAT] [--ttur BOOL] [--gamma FLOAT] [--nkimg INT]
                [--kimg INT] [--topk FLOAT] [--aug {noaug,ada,fixed,adarv}]
                [--p FLOAT] [--target TARGET] [--initstrength INITSTRENGTH]
                [--augpipe {blit,geom,color,filter,noise,cutout,bg,bgc,bgcf,bgcfn,bgcfnc}]
                [--cmethod {nocmethod,bcr,zcr,pagan,wgangp,auxrot,spectralnorm,shallowmap,adropout}]
                [--dcap FLOAT] [--resume RESUME] [--freezed INT]

Train a GAN using the techniques described in the paper
"Training Generative Adversarial Networks with Limited Data".

optional

In [ ]:
training_path = project_path / 'training' / dataset_name
if not training_path.is_dir():
    %mkdir "{training_path}"

#how often should the model generate samples and a .pkl file
snapshot_count = 2
#should the images be mirrored left to right?
mirrored = True
#should the images be mirrored top to bottom?
mirroredY = False
#metrics?
metric_list = None
#augments
augs = 'bgc'

resume_from = 'noresume'

!python "{stylegan2_repo_path / 'train.py'}" --outdir="{training_path}" \
    --data="{local_dataset_path}" --resume="{resume_from}" \
    --snap={snapshot_count} --augpipe={augs} \
    --mirror={mirrored} --mirrory={mirroredY} --cfg={'auto'} \
    --metrics={metric_list} #--dry-run

tcmalloc: large alloc 4294967296 bytes == 0x55edabd04000 @  0x7fe6f1f84001 0x7fe6ef18754f 0x7fe6ef1d7b58 0x7fe6ef1dbb17 0x7fe6ef27a203 0x55eda5125544 0x55eda5125240 0x55eda5199627 0x55eda5193ced 0x55eda512748c 0x55eda5168159 0x55eda51650a4 0x55eda5125d49 0x55eda519994f 0x55eda51939ee 0x55eda5065e2b 0x55eda5195fe4 0x55eda51939ee 0x55eda5065e2b 0x55eda5195fe4 0x55eda5193ced 0x55eda5065e2b 0x55eda5195fe4 0x55eda5126afa 0x55eda5194915 0x55eda51939ee 0x55eda51936f3 0x55eda525d4c2 0x55eda525d83d 0x55eda525d6e6 0x55eda5235163
tcmalloc: large alloc 4294967296 bytes == 0x55eeabd04000 @  0x7fe6f1f821e7 0x7fe6ef18746e 0x7fe6ef1d7c7b 0x7fe6ef1d835f 0x7fe6ef27a103 0x55eda5125544 0x55eda5125240 0x55eda5199627 0x55eda51939ee 0x55eda5126bda 0x55eda5195737 0x55eda51939ee 0x55eda5126bda 0x55eda5195737 0x55eda51939ee 0x55eda5126bda 0x55eda5195737 0x55eda5126afa 0x55eda5194915 0x55eda51939ee 0x55eda5126bda 0x55eda5198d00 0x55eda51939ee 0x55eda5126bda 0x55eda5195737 0x55eda5193ced 0x55eda512748c 0x55eda516

## Generate images from pre-trained model

You can finally generate images using a pre-trained network once everything is set-up. You can naturally use [your own model once it is trained](#scrollTo=Ti11YiPAiQpb&uniqifier=1) or use the ones NVLab published on [their website](https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/).

<p align="center">
    <img
    alt="Night Sky Latent Walk"
    width="450" height="300"
    src="https://github.com/ArthurFDLR/GANightSky/blob/main/.github/Random_Generation.png?raw=true">
</p>

In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-images --help 

usage: generate.py generate-images [-h] --network NETWORK_PKL --seeds SEEDS
                                   [--trunc TRUNCATION_PSI]
                                   [--class CLASS_IDX] [--create-grid]
                                   [--outdir DIR] [--save_vector] [--fixnoise]
                                   [--jpg_quality JPG_QUALITY]

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --seeds SEEDS         List of random seeds
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --class CLASS_IDX     Class label (default: unconditional)
  --create-grid         Add flag to save the generated images in a grid
  --outdir DIR          Root directory for run results (default: out)
  --save_vector         also save vector in .npy format
  --fixnoise            generate images using fixed noise (more accurate for
                        interpolations)
  

In [ ]:
from numpy import random
seed_init = random.randint(10000)
nbr_images = 100

generation_from = '/content/drive/MyDrive/StyleGAN2-ADA/training/dr_cyz/00000-tfr-mirror-auto1-bgc-noresume-256_de_curto_and_de_zarza/network-snapshot-000798.pkl'
!python "{stylegan2_repo_path / 'generate.py'}" generate-images \
    --outdir="{project_path / 'out' / 'dr_cyz_256_100'}" --trunc=0.7 \
    --seeds={seed_init}-{seed_init+nbr_images-1} --create-grid \
    --network={generation_from}

Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/dr_cyz/00000-tfr-mirror-auto1-bgc-noresume-256_de_curto_and_de_zarza/network-snapshot-000798.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 5269 (0/100) ...
Generating image for seed 5270 (1/100) ...
Generating image for seed 5271 (2/100) ...
Generating image for seed 5272 (3/100) ...
Generating image for seed 5273 (4/100) ...
Generating image for seed 5274 (5/100) ...
Generating image for seed 5275 (6/100) ...
Generating image for seed 5276 (7/100) ...
Generating image for seed 5277 (8/100) ...
Generating image for seed 5278 (9/100) ...
Generating image for seed 5279 (10/100) ...
Generating image for seed 5280 (11/100) ...
Generating image for seed 5281 (12/100) ...
Generating image for seed 5282 (13/100) ...
Generating image for seed 5283 (14/100) ...
Generating image for seed 5284 (15/100) ...
Genera

In [ ]:
#Run this cell to compute FID metric data, if desired.

generation_from = '/content/drive/MyDrive/StyleGAN2-ADA/training/dr_cyz/00000-tfr-mirror-auto1-bgc-noresume-256_de_curto_and_de_zarza/network-snapshot-000798.pkl'
!python "{stylegan2_repo_path / 'calc_metrics.py'}" \
    --metrics=fid50k_full --metricdata="{local_dataset_path}" --mirror=1 --network={generation_from}   

Loading network from "/content/drive/MyDrive/StyleGAN2-ADA/training/dr_cyz/00000-tfr-mirror-auto1-bgc-noresume-256_de_curto_and_de_zarza/network-snapshot-000798.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Compiling... Loading... Done.

Gs                            Params    OutputShape         WeightShape     
---                           ---       ---                 ---             
latents_in                    -         (?, 512)            -               
labels_in                     -         (?, 0)              -               
G_mapping/Normalize           -         (?, 512)            -               
G_mapping/Dense0              262656    (?, 512)            (512, 512)      
G_mapping/Dense1              262656    (?, 512)            (512, 512)      
G_mapping/Broadcast           -         (?, 14, 512)        -               
dlatent_avg                   -         (512,)            

## Latent space exploration

It is also possible to explore the latent space associated with our model and [generate videos like this one](https://youtu.be/dcb4Ckpkx2o).


In [ ]:
%pip install opensimplex
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --help 

usage: generate.py generate-latent-walk [-h] --network NETWORK_PKL
                                        [--trunc TRUNCATION_PSI]
                                        [--walk-type WALK_TYPE]
                                        [--frames FRAMES] [--fps FRAMERATE]
                                        [--seeds SEEDS] [--npys NPYS]
                                        [--save_vector] [--diameter DIAMETER]
                                        [--start_seed START_SEED]
                                        [--outdir DIR]

optional arguments:
  -h, --help            show this help message and exit
  --network NETWORK_PKL
                        Network pickle filename
  --trunc TRUNCATION_PSI
                        Truncation psi (default: 0.5)
  --walk-type WALK_TYPE
                        Type of walk (default: line)
  --frames FRAMES       Frame count (default: 240
  --fps FRAMERATE       Starting value
  --seeds SEEDS         List of random seeds
  --npys NPYS       

In [ ]:
from numpy import random
walk_types = ['line', 'sphere', 'noiseloop', 'circularloop']
latent_walk_path = project_path / 'out' / 'latent_walk_sphere_cyz_256'
if not latent_walk_path.is_dir():
    %mkdir "{latent_walk_path}"

explored_network = '/content/drive/MyDrive/StyleGAN2-ADA/training/dr_cyz/00000-tfr-mirror-auto1-bgc-noresume-256_de_curto_and_de_zarza/network-snapshot-000798.pkl'

seeds = [random.randint(10000) for i in range(10)]
print(','.join(map(str, seeds)))
print("Base seeds:", seeds)
!python "{stylegan2_repo_path / 'generate.py'}" generate-latent-walk --network="{explored_network}" \
    --outdir="{latent_walk_path}" --trunc=0.7 --walk-type="{walk_types[1]}" \
    --seeds={','.join(map(str, seeds))} --frames {len(seeds)*20}

1063,5112,8352,7866,2074,7117,2999,2892,2821,7171
Base seeds: [1063, 5112, 8352, 7866, 2074, 7117, 2999, 2892, 2821, 7171]
Loading networks from "/content/drive/MyDrive/StyleGAN2-ADA/training/dr_cyz/00000-tfr-mirror-auto1-bgc-noresume-256_de_curto_and_de_zarza/network-snapshot-000798.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
slerp
Generating image for step 0/207 ...
Generating image for step 1/207 ...
Generating image for step 2/207 ...
Generating image for step 3/207 ...
Generating image for step 4/207 ...
Generating image for step 5/207 ...
Generating image for step 6/207 ...
Generating image for step 7/207 ...
Generating image for step 8/207 ...
Generating image for step 9/207 ...
Generating image for step 10/207 ...
Generating image for step 11/207 ...
Generating image for step 12/207 ...
Generating image for step 13/207 ...
Generating image for step 14/207 ...
Generating image for step 

## While you wait ...

... learn more about Generative Adversarial Networks and StyleGAN2-ADA:

*   [This Night Sky Does Not Exist](https://arthurfindelair.com/thisnightskydoesnotexist/): Generation of images from a model created using this Notebook on Google Colab Pro.
*   [This **X** Does Not Exist](https://thisxdoesnotexist.com/): Collection of sites showing the power of GANs.
*   [Karras, Tero, et al. _Analyzing and Improving the Image Quality of StyleGAN._ CVPR 2020.](https://arxiv.org/pdf/2006.06676.pdf): Paper published for the release of StyleGAN2-ADA.
*   [Official implementation of StyleGAN2-ADA](https://github.com/NVlabs/stylegan2-ada)
*   [StyleGAN v2: notes on training and latent space exploration](https://towardsdatascience.com/stylegan-v2-notes-on-training-and-latent-space-exploration-e51cf96584b3): Interesting article from Toward Data Science